In [5]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [57]:
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    if isinstance(url,str):
        response = requests.get(url)
    else:
        #if already it is a loaded html page
        response = url
    if response is not None:
        html = BeautifulSoup(response.text, 'html.parser')
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
        if search:
            soup = html            
            r = ''
            if 'find' in search.keys():
                print('finding',search['find'])
                soup = soup.find(**search['find'])
                r = soup
            if 'find_all' in search.keys():
                print('finding all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
        
        return r

In [70]:
#getting the africa leaders
url = "https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/"
res_gov = get_elements(url, search={'find_all':{'class_':'wp-block-embed__wrapper'}}) 
res_gv = get_elements(url, search={'find_all':{'class_':'twitter-tweet'}})
officials = []
for i in range(2 ,len(res_gv)):
    mainblk = res_gv[i]
    olist = mainblk.find_all(text=True , recursive=False)
    officials.append(olist)
final = []
for item in officials:
    final.append(item[0])

finallist = pd.DataFrame(final, columns=['User'])
top10 = finallist.head(10)
top10.to_csv('africaleaders.csv',index=True )

finding all of {'class_': 'wp-block-embed__wrapper'}
finding all of {'class_': 'twitter-tweet'}


In [87]:
#getting africa freak data 
def scrape(url):
	r = requests.get(url)
	soup = BeautifulSoup(r.content , features="html.parser")
	return soup

url = "https://africafreak.com/100-most-influential-twitter-users-in-africa"
influencers = scrape(url)
names = influencers.findAll("h2")
fulllist = []
for name in names:
    thisname = name.text
    namenno = thisname.split(".")
    fulllist.append(namenno)
influencerrank = pd.DataFrame(fulllist , columns=['Rank', 'User', 'Tag'])
highest10 = influencerrank.tail(10)
highest10.to_csv('top_influencers.csv', index=False)

ValueError: 2 columns passed, passed data had 3 columns